# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb7b4cd7460>
├── 'a' --> tensor([[-0.5522,  0.4539, -1.2755],
│                   [ 0.9862, -0.1831,  1.4673]])
└── 'x' --> <FastTreeValue 0x7fb7b4cd77f0>
    └── 'c' --> tensor([[-0.9148,  1.6883, -0.0744,  2.1475],
                        [ 0.1008, -1.0256,  0.6670,  0.2197],
                        [-0.2459, -2.1217, -0.7434, -1.5399]])

In [4]:
t.a

tensor([[-0.5522,  0.4539, -1.2755],
        [ 0.9862, -0.1831,  1.4673]])

In [5]:
%timeit t.a

59.8 ns ± 0.0363 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb7b4cd7460>
├── 'a' --> tensor([[-1.3322,  1.1355, -0.7482],
│                   [-1.7904, -0.7958,  1.5811]])
└── 'x' --> <FastTreeValue 0x7fb7b4cd77f0>
    └── 'c' --> tensor([[-0.9148,  1.6883, -0.0744,  2.1475],
                        [ 0.1008, -1.0256,  0.6670,  0.2197],
                        [-0.2459, -2.1217, -0.7434, -1.5399]])

In [7]:
%timeit t.a = new_value

60.5 ns ± 0.0246 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5522,  0.4539, -1.2755],
               [ 0.9862, -0.1831,  1.4673]]),
    x: Batch(
           c: tensor([[-0.9148,  1.6883, -0.0744,  2.1475],
                      [ 0.1008, -1.0256,  0.6670,  0.2197],
                      [-0.2459, -2.1217, -0.7434, -1.5399]]),
       ),
)

In [10]:
b.a

tensor([[-0.5522,  0.4539, -1.2755],
        [ 0.9862, -0.1831,  1.4673]])

In [11]:
%timeit b.a

54.6 ns ± 0.0261 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.8979, -0.0206, -0.1976],
               [ 1.1287,  1.1737, -0.1409]]),
    x: Batch(
           c: tensor([[-0.9148,  1.6883, -0.0744,  2.1475],
                      [ 0.1008, -1.0256,  0.6670,  0.2197],
                      [-0.2459, -2.1217, -0.7434, -1.5399]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.203 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

851 ns ± 4.03 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 14.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 581 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 358 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb710d5b310>
├── 'a' --> tensor([[[-0.5522,  0.4539, -1.2755],
│                    [ 0.9862, -0.1831,  1.4673]],
│           
│                   [[-0.5522,  0.4539, -1.2755],
│                    [ 0.9862, -0.1831,  1.4673]],
│           
│                   [[-0.5522,  0.4539, -1.2755],
│                    [ 0.9862, -0.1831,  1.4673]],
│           
│                   [[-0.5522,  0.4539, -1.2755],
│                    [ 0.9862, -0.1831,  1.4673]],
│           
│                   [[-0.5522,  0.4539, -1.2755],
│                    [ 0.9862, -0.1831,  1.4673]],
│           
│                   [[-0.5522,  0.4539, -1.2755],
│                    [ 0.9862, -0.1831,  1.4673]],
│           
│                   [[-0.5522,  0.4539, -1.2755],
│                    [ 0.9862, -0.1831,  1.4673]],
│           
│                   [[-0.5522,  0.4539, -1.2755],
│                    [ 0.9862, -0.1831,  1.4673]]])
└── 'x' --> <FastTreeValue 0x7fb7099f81f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 47.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb7105f0bb0>
├── 'a' --> tensor([[-0.5522,  0.4539, -1.2755],
│                   [ 0.9862, -0.1831,  1.4673],
│                   [-0.5522,  0.4539, -1.2755],
│                   [ 0.9862, -0.1831,  1.4673],
│                   [-0.5522,  0.4539, -1.2755],
│                   [ 0.9862, -0.1831,  1.4673],
│                   [-0.5522,  0.4539, -1.2755],
│                   [ 0.9862, -0.1831,  1.4673],
│                   [-0.5522,  0.4539, -1.2755],
│                   [ 0.9862, -0.1831,  1.4673],
│                   [-0.5522,  0.4539, -1.2755],
│                   [ 0.9862, -0.1831,  1.4673],
│                   [-0.5522,  0.4539, -1.2755],
│                   [ 0.9862, -0.1831,  1.4673],
│                   [-0.5522,  0.4539, -1.2755],
│                   [ 0.9862, -0.1831,  1.4673]])
└── 'x' --> <FastTreeValue 0x7fb71068d490>
    └── 'c' --> tensor([[-0.9148,  1.6883, -0.0744,  2.1475],
                        [ 0.1008, -1.0256,  0.6670,  0.2197],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 72.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.8 µs ± 56.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.5522,  0.4539, -1.2755],
                [ 0.9862, -0.1831,  1.4673]],
       
               [[-0.5522,  0.4539, -1.2755],
                [ 0.9862, -0.1831,  1.4673]],
       
               [[-0.5522,  0.4539, -1.2755],
                [ 0.9862, -0.1831,  1.4673]],
       
               [[-0.5522,  0.4539, -1.2755],
                [ 0.9862, -0.1831,  1.4673]],
       
               [[-0.5522,  0.4539, -1.2755],
                [ 0.9862, -0.1831,  1.4673]],
       
               [[-0.5522,  0.4539, -1.2755],
                [ 0.9862, -0.1831,  1.4673]],
       
               [[-0.5522,  0.4539, -1.2755],
                [ 0.9862, -0.1831,  1.4673]],
       
               [[-0.5522,  0.4539, -1.2755],
                [ 0.9862, -0.1831,  1.4673]]]),
    x: Batch(
           c: tensor([[[-0.9148,  1.6883, -0.0744,  2.1475],
                       [ 0.1008, -1.0256,  0.6670,  0.2197],
                       [-0.2459, -2.1217, -0.7434, -1.5399]],
         

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.5522,  0.4539, -1.2755],
               [ 0.9862, -0.1831,  1.4673],
               [-0.5522,  0.4539, -1.2755],
               [ 0.9862, -0.1831,  1.4673],
               [-0.5522,  0.4539, -1.2755],
               [ 0.9862, -0.1831,  1.4673],
               [-0.5522,  0.4539, -1.2755],
               [ 0.9862, -0.1831,  1.4673],
               [-0.5522,  0.4539, -1.2755],
               [ 0.9862, -0.1831,  1.4673],
               [-0.5522,  0.4539, -1.2755],
               [ 0.9862, -0.1831,  1.4673],
               [-0.5522,  0.4539, -1.2755],
               [ 0.9862, -0.1831,  1.4673],
               [-0.5522,  0.4539, -1.2755],
               [ 0.9862, -0.1831,  1.4673]]),
    x: Batch(
           c: tensor([[-0.9148,  1.6883, -0.0744,  2.1475],
                      [ 0.1008, -1.0256,  0.6670,  0.2197],
                      [-0.2459, -2.1217, -0.7434, -1.5399],
                      [-0.9148,  1.6883, -0.0744,  2.1475],
                      [ 0.1008, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

317 µs ± 626 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
